In [2]:
import pandas as pd

In [2]:
table=pd.read_csv('/home/ls/rachelcw/projects/SPOT/spot/example_data/exon_exon_junction_file.txt', sep=' ')

In [6]:
# create list of blood samples from gtex #

# annot=pd.read_csv("/data01/private/projects/splicing_rd/GTEx/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt",sep="\t")
# annot_blood=annot.loc[annot["SMTSD"]=="Whole Blood"]

# annot_blood["SAMPID"].to_csv("~/projects/SPOT/sample_blood_gtex.20230319.txt",sep="\t", index=0)

Intergration leafcutter results (next step with spot)

In [89]:
# # cluster_significance # #
cluster=pd.read_csv("/home/ls/rachelcw/projects/rare_disease/data/pilot1_ds_cluster_significance.txt",sep="\t",usecols=["cluster","p.adjust"])
cluster[["chr","cluster"]]=cluster['cluster'].str.split(':', expand=True)


In [90]:
# # effect sizes # #
ds=pd.read_csv("/home/ls/rachelcw/projects/rare_disease/data/pilot1_ds_effect_sizes.txt",sep="\t",usecols=["intron","deltapsi","patient","healthy"])
ds[['chr', 'start', 'end','cluster']] = ds['intron'].str.split(':', expand=True)
ds.drop(['intron'], axis=1, inplace=True)

In [92]:
table=ds.merge(cluster, on=["chr","cluster"], how="left")
table[['cluster','strand']]=table['cluster'].str.rsplit('_', expand=True, n=1)
table["strand"]="+"
# table.dropna(axis=0)

In [93]:
import pyensembl


In [94]:

data = pyensembl.Genome(
    reference_name="GRCh38",
    annotation_name="GRCh38",
    gtf_path_or_url="/private/resources/gencode.v34.GRCh38.annotation.gtf")

data.index()
# quay.io/biocontainers/pyensembl:2.1.0--pyh7cba7a3_0

In [96]:
table["gene"]=pd.NA

# how to relate to mitochondrial chromosome
for index, row in table.iterrows():
    gene_name = data.gene_names_at_locus(contig=row["chr"], position=int(row["start"]),end=int(row["end"]), strand=row["strand"])
    if gene_name != []:
        table.at[index,"gene"]=gene_name

In [99]:
table[["chr","start","end","gene","strand","healthy","patient","deltapsi","p.adjust"]].to_csv("/home/ls/rachelcw/projects/rare_disease/data/leafcutter_results.bed",sep="\t", index=0)

convert leafcutter results to bed format

In [5]:
cluster=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/lc_20230108/lc_20230108_perind_numers.counts.gz", sep=" ")

In [22]:
ds=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/ds.a1.20230115_effect_sizes.txt", sep="\t", usecols=["intron"])

In [23]:
ds[['chrom', 'start', 'end','clu']] = ds['intron'].str.split(':', expand=True)

# Drop columns
ds.drop(['intron','cluster'], axis=1, inplace=True)

# Write the BED file
ds.to_csv('/home/ls/rachelcw/projects/rare_disease/output_file.bed', sep='\t', index=False, header=False)